In [66]:
import numpy as np
import matplotlib.pyplot as plt
import random
import pandas as pd

In [67]:
data = pd.read_csv("mnist_train.csv")
test = pd.read_csv("mnist_test.csv")

In [68]:
test.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [69]:
data = np.array(data)
test = np.array(test)
m,n = data.shape
labels = data[:,0]
values = data[:,1:n]
test_labels = test[:,0]
test_values = test[:,1:n]

In [94]:
class Layer:
    def __init__(self):
        self.inputNum = None
        self.outputNum = None

class Dense(Layer):
    def __init__(self,inputNum,outputNum):
        self.inputNum = inputNum
        self.outputNum = outputNum
        self.weigths = np.random.uniform(-0.5,0.5,(self.outputNum,self.inputNum))
        self.bias = np.random.uniform(-0.5,0.5,(self.outputNum,1))

    def forward(self,inputs):
        self.inputs = inputs
        return np.dot(self.weigths,inputs) + self.bias
    
    def backward(self,outputsGradient,learningRate):
        weights_gradient = np.dot(outputsGradient,self.inputs.T)
        self.weigths -= learningRate*weights_gradient
        self.bias -= learningRate*outputsGradient
        return np.dot(self.weigths.T,outputsGradient)
    

class NeuralNetwork():
    def __init__(self,layers:list[Layer])->list[int]:
        self.layers = layers

    def cross_entropy(self, y_true, y_pred):
        return -np.sum(y_true * np.log(y_pred))

    def cross_entropy_prime(self, y_true, y_pred):      
        return - (y_true / y_pred)

    
    def verifyAnswer(self,answer,prediction):
        return prediction.argmax() == answer
    
    def oneHotEncode(self,value):
        vector = np.zeros((10, 1))
        vector[value, 0] = 1
        return vector

    def calculateError(self,predictedVector,correctVector):
        return predictedVector-correctVector



    def batch_train(self,dataset_values,dataset_labels, epochs, learningRate):
        for epoch in range(epochs):
            correct = 0
            for img,answer in zip(dataset_values,dataset_labels):
                desiredOutput = self.oneHotEncode(answer)
                currentImg = np.reshape(img,(n-1,1))
                input = currentImg
                for i,layer in enumerate(self.layers):
                    input = layer.forward(input)
                    # print(f"Camada numero {i}: {input}")
                correct += 1 if self.verifyAnswer(answer,input) else 0
                grad = self.calculateError(input,desiredOutput)
            for layer in reversed(self.layers):
                grad = layer.backward(grad,learningRate)
            print(f"Epoch {epoch} --> Accuracy = {correct/len(dataset_labels)*100:.2f}%")

In [89]:
class Activation(Layer):
    def function(self,inputs):
        pass
    def function_prime(self,inputs):
        pass

    def forward(self,inputs):
        self.inputs = inputs
        return self.function(self.inputs)

    def backward(self,outputGradient,learningRate):
        return np.multiply(self.function_prime(self.inputs),outputGradient)
    

class Sigmoid(Activation):
    def function(self,inputs):
        return 1/(1+np.exp(-inputs))
    
    def function_prime(self,inputs):
        temp = self.function(inputs)
        return temp*(1-temp)

class Relu(Activation):
    def __init__(self):
        super().__init__()

    def function(self, inputs):
        return np.maximum(0,inputs)
    
    def function_prime(self, inputs):
        return inputs > 0


class Softmax(Activation):
    
    def function(self,input):
        tmp = np.exp(input)
        self.output = tmp/np.sum(tmp)
        return self.output

    def function_prime(self, inputs):
        n = np.size(self.output)
        tmp = np.tile(self.output,n)
        print((np.identity(n)-np.transpose(tmp)))
        return tmp*(np.identity(n)-np.transpose(tmp))



class Tanh(Activation):
    def __init__(self):
        self.tanh = lambda x: np.tanh(x)
        self.tanh_prime = lambda x: 1- np.tanh(x)**2
        super().__init__()
        
    def forward(self,inputs):
        self.inputs = inputs
        return self.tanh(self.inputs)

    def backward(self,outputGradient,learningRate):
        return np.multiply(outputGradient,self.tanh_prime(self.inputs))

In [100]:

network = NeuralNetwork([
    Dense(784,10),
    Tanh(),
    Dense(10,10),
    Tanh(),
])

In [101]:
network.batch_train(values,labels,epochs=20,learningRate=0.1)

Epoch 0 --> Accuracy = 6.98%
Epoch 1 --> Accuracy = 6.99%
Epoch 2 --> Accuracy = 6.58%
Epoch 3 --> Accuracy = 6.42%
Epoch 4 --> Accuracy = 6.48%
Epoch 5 --> Accuracy = 6.43%
Epoch 6 --> Accuracy = 6.43%
Epoch 7 --> Accuracy = 6.43%
Epoch 8 --> Accuracy = 6.43%
Epoch 9 --> Accuracy = 6.43%
Epoch 10 --> Accuracy = 6.43%
Epoch 11 --> Accuracy = 6.49%
Epoch 12 --> Accuracy = 6.49%
Epoch 13 --> Accuracy = 6.49%
Epoch 14 --> Accuracy = 6.49%
Epoch 15 --> Accuracy = 6.49%
Epoch 16 --> Accuracy = 6.49%
Epoch 17 --> Accuracy = 6.49%
Epoch 18 --> Accuracy = 6.49%
Epoch 19 --> Accuracy = 6.49%


In [9]:
correct = 0
for num in range(len(test_labels)):
    input = np.reshape(values[num],(n-1,1))
    for layer in network.layers:
        input = layer.forward(input)
    if input.argmax() == labels[num]:
        correct+=1
print(f"porcentagem de acerto: {correct/len(test_labels)*100:.6f}%")

porcentagem de acerto: 10.010000%


In [99]:
# class Layer:
#     def __init__(self):
#         self.input = None
#         self.output = None

#     def foward(self,input):
#         pass

#     def backward(self,output_gradient,learning_rate):
#         pass

# class Dense(Layer):
#     def __init__(self,input_size,output_size):
#         self.weights = np.random.randn(output_size,input_size)
#         self.bias = np.random.randn(output_size,1)

#     def foward(self, input):
#         self.input = input
#         return np.dot(self.weights,self.input) + self.bias
    
#     def backward(self, output_gradient, learning_rate):
#         weights_gradient = np.dot(output_gradient,self.input.T)
#         self.weights -= learning_rate*weights_gradient
#         self.bias -= learning_rate*output_gradient
#         return np.dot(self.weights.T,output_gradient)

class Activation(Layer):
    def __init__(self,activation,acticvation_prime):
        self.activation = activation
        self.activation_prime = acticvation_prime

    def forward(self, input):
        self.input = input
        return self.activation(self.input)
    
    def backward(self,output_gradient, learningRate):
        return np.multiply(output_gradient,self.activation_prime(self.input))


class Tanh(Activation):
    def __init__(self):
        tanh = lambda x: np.tanh(x)
        tanh_prime = lambda x: 1- np.tanh(x)**2
        super().__init__(tanh,tanh_prime)